The aim of this code is to take an inpur level 2 OLCI file and create a tri-stimulus true colour image from the reflectance values.

Then it will also creat a matched image from the level 1 data allowing the masked data to also be viewed.  By comparing these 2 images you can look at the quality of the masking that is being applied to the OLCI data between level 1 and level 2.

The example image has been chosen as it contains sea-ice, clouds, coastal waters and land.  This means that there are a variety of masks being applied between level 1 and level 2.

Creating a 'true colour' image using both the level 1 radiances and the level 2 reflectances allows you to see the effect of atmospheric correction (the difference between the ocean colour as seen at the top of atmosphere and at the surface).

Import a number of required modules and functions to aid in the processing

In [4]:
# imports
import shutil, os, sys
import argparse
import numpy as np
import matplotlib.pyplot as plt
import urllib.request

# ------------------------------------------------------------------------------
# snappy modules
import snappy
from snappy import Product, ProductData, ProductIO, ProductUtils, ProgressMonitor


Now we define that input data from level 2 (as stated earlier the image here is chosen to demonstrate multiple flags but you could change this to any other level 2 data files for a region of your own interest).

In [5]:
indir='/data/datasets/operational/sentinel3a_olci/level2/current/non_time_critical/swath/0d/2017/07/05/safe/S3A_OL_2_WFR____20170705T063341_20170705T063641_20170706T150012_0179_019_291_1799_MAR_O_NT_002.SEN3'

COPY_DIR = indir.replace('.SEN3', '_copy.SEN3')
COPY_DIR= '/tmp/'+os.path.basename(COPY_DIR)
if os.path.isdir(COPY_DIR):
    print COPY_DIR+' already exists'
else:
    shutil.copytree(indir, COPY_DIR)


From the input file we will define the required data for input to the tri-stimulus equation.

In [6]:
SOURCE_PRODUCT = ProductIO.readProduct(COPY_DIR+'/xfdumanifest.xml')

IMAGE_MANAGER = snappy.jpy.get_type('org.esa.snap.core.image.ImageManager')
JAI = snappy.jpy.get_type('javax.media.jai.JAI')
Oa01_reflectance=SOURCE_PRODUCT.getBand('Oa01_reflectance')
Oa02_reflectance=SOURCE_PRODUCT.getBand('Oa02_reflectance')
Oa03_reflectance=SOURCE_PRODUCT.getBand('Oa03_reflectance')
Oa04_reflectance=SOURCE_PRODUCT.getBand('Oa04_reflectance')
Oa05_reflectance=SOURCE_PRODUCT.getBand('Oa05_reflectance')
Oa06_reflectance=SOURCE_PRODUCT.getBand('Oa06_reflectance')
Oa07_reflectance=SOURCE_PRODUCT.getBand('Oa07_reflectance')
Oa08_reflectance=SOURCE_PRODUCT.getBand('Oa08_reflectance')
Oa09_reflectance=SOURCE_PRODUCT.getBand('Oa09_reflectance')
Oa10_reflectance=SOURCE_PRODUCT.getBand('Oa10_reflectance')




Snappy allows the definition of an equation without reading all the data into python.  The epxression can then be executed using the snap processing and will make use of features such as chunking.  First we define the expressions for each of the tri-stimulus rgb bands that will go into a 'true colour' image.

In [7]:
Rexpression = 'log(0.05 + 0.01 * Oa01_reflectance + 0.09 * Oa02_reflectance + 0.35 * Oa03_reflectance + 0.04 * Oa04_reflectance + 0.01 * Oa05_reflectance + 0.59 * Oa06_reflectance + 0.85 * Oa07_reflectance + 0.12 * Oa08_reflectance + 0.07 * Oa09_reflectance + 0.04 * Oa10_reflectance)'
Gexpression = 'log(0.05 + 0.26 * Oa03_reflectance + 0.21 * Oa04_reflectance + 0.50 * Oa05_reflectance + Oa06_reflectance + 0.38 * Oa07_reflectance + 0.04 * Oa08_reflectance + 0.03 * Oa09_reflectance + 0.02 * Oa10_reflectance)'
Bexpression = 'log(0.05 + 0.07 * Oa01_reflectance + 0.28 * Oa02_reflectance + 1.77 * Oa03_reflectance + 0.47 * Oa04_reflectance + 0.16 * Oa05_reflectance)'

In [8]:
newband1 = SOURCE_PRODUCT.addBand('red', Rexpression)
newband1.setDescription("red vis band from tristimulus")
newband2 = SOURCE_PRODUCT.addBand('green', Gexpression)
newband2.setDescription("green vis band from tristimulus")
newband3 = SOURCE_PRODUCT.addBand('blue', Bexpression)
newband3.setDescription("blue vis band from tristimulus")
    

Now we create an RGB image from the red green and blue bands created via the tri-stimulus expressions.

In [9]:

RED = SOURCE_PRODUCT.getBand('red')
GREEN = SOURCE_PRODUCT.getBand('green')
BLUE = SOURCE_PRODUCT.getBand('blue')


img_info = ProductUtils.createImageInfo([RED,GREEN, BLUE], True, ProgressMonitor.NULL)
img = IMAGE_MANAGER.getInstance().createColoredBandImage([RED, GREEN, BLUE], img_info, 0)



This true colour image can now be written out to a png for viewing.

In [10]:
filename='/tmp/OLCI_test_out_tri_lvl2.png'
file_format='PNG'

JAI.create("filestore", img, filename, file_format)

javax.media.jai.RenderedOp(objectRef=0x562c5dd40e38)

You can view an image inline in the workbook but if it is a very large image it will probably exceed a memory limit for console input/output. To view the png here, uncomment the second line below (Image...) but don't be surprised if you get an error message.

In [11]:
from IPython.display import Image
#Image(filename='/tmp/OLCI_test_out_tri_lvl2.png')

After p[rocessing we will cleanup the copied data used for processing the image so that we are not retaining data duplicates and inflating data volumes unnecessarily.

In [12]:
#cleanup 
shutil.rmtree(COPY_DIR)



We will now repeat the entire process using a level 1 file.  Note that now we are using radiances rather than reflectances and the expressions for the tri-stimulus bands have changed slightly.

In [13]:
infile_location='https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/450/S3A_OL_1_EFR/2017/186/S3A_OL_1_EFR____20170705T063341_20170705T063641_20180505T200006_0179_019_291_1800_LR2_R_NT_002.zip'

urllib.request.urlretrieve(infile_location, "mp3.mp3")

COPY_DIR = indir.replace('.SEN3', '_copy.SEN3')
COPY_DIR= '/tmp/'+os.path.basename(COPY_DIR)
if os.path.isdir(COPY_DIR):
    print COPY_DIR+' already exists'
else:
    shutil.copytree(indir, COPY_DIR)
SOURCE_PRODUCT = ProductIO.readProduct(COPY_DIR+'/xfdumanifest.xml')

Oa01_radiance=SOURCE_PRODUCT.getBand('Oa01_radiance')
Oa02_radiance=SOURCE_PRODUCT.getBand('Oa02_radiance')
Oa03_radiance=SOURCE_PRODUCT.getBand('Oa03_radiance')
Oa04_radiance=SOURCE_PRODUCT.getBand('Oa04_radiance')
Oa05_radiance=SOURCE_PRODUCT.getBand('Oa05_radiance')
Oa06_radiance=SOURCE_PRODUCT.getBand('Oa06_radiance')
Oa07_radiance=SOURCE_PRODUCT.getBand('Oa07_radiance')
Oa08_radiance=SOURCE_PRODUCT.getBand('Oa08_radiance')
Oa09_radiance=SOURCE_PRODUCT.getBand('Oa09_radiance')
Oa10_radiance=SOURCE_PRODUCT.getBand('Oa10_radiance')

Rexpression = 'log(1.0 + 0.01 * Oa01_radiance + 0.09 * Oa02_radiance + 0.35 * Oa03_radiance + 0.04 * Oa04_radiance + 0.01 * Oa05_radiance + 0.59 * Oa06_radiance + 0.85 * Oa07_radiance + 0.12 * Oa08_radiance + 0.07 * Oa09_radiance + 0.04 * Oa10_radiance)'
Gexpression = 'log(1.0 + 0.26 * Oa03_radiance + 0.21 * Oa04_radiance + 0.50 * Oa05_radiance + Oa06_radiance + 0.38 * Oa07_radiance + 0.04 * Oa08_radiance + 0.03 * Oa09_radiance + 0.02 * Oa10_radiance)'
Bexpression = 'log(1.0 + 0.07 * Oa01_radiance + 0.28 * Oa02_radiance + 1.77 * Oa03_radiance + 0.47 * Oa04_radiance + 0.16 * Oa05_radiance)'

newband1 = SOURCE_PRODUCT.addBand('red', Rexpression)
newband1.setDescription("red vis band from tristimulus")
newband2 = SOURCE_PRODUCT.addBand('green', Gexpression)
newband2.setDescription("green vis band from tristimulus")
newband3 = SOURCE_PRODUCT.addBand('blue', Bexpression)
newband3.setDescription("blue vis band from tristimulus")
  
RED = SOURCE_PRODUCT.getBand('red')
GREEN = SOURCE_PRODUCT.getBand('green')
BLUE = SOURCE_PRODUCT.getBand('blue')
  
img_info = ProductUtils.createImageInfo([RED,GREEN, BLUE], True, ProgressMonitor.NULL)
img = IMAGE_MANAGER.getInstance().createColoredBandImage([RED, GREEN, BLUE], img_info, 0)

filename='/tmp/OLCI_test_out_tri_lvl1.png'
JAI.create("filestore", img, filename, file_format)

#cleanup 
shutil.rmtree(COPY_DIR)

